In [1]:
import pandas as pd
from openai import AzureOpenAI
from pinecone import Pinecone, ServerlessSpec
client = AzureOpenAI(
  api_key = "d70b34fbd24d4016a5cf88dbc5f91e78",  
  api_version = "2023-05-15",
  azure_endpoint ="https://openai-mais-2.openai.azure.com/" 
)
# connect pinecone
index_name = 'semantic-search-openai'
pc = Pinecone(api_key='b5d40c2b-abda-4590-8cb5-06251507c483')
# connect to index
index = pc.Index(index_name)

In [3]:
def get_embedding(text, model="test-embedding-ada-002"): 
    return client.embeddings.create(input = [text], model=model).data[0].embedding
# x = get_embedding('test embeddings')
# print(x[0:10])
# print(len(x))

In [6]:
##### search document data with openai embedding
def search_docs(user_query, top=5):
    xq = get_embedding(
        user_query,
        model="test-embedding-ada-002" # engine should be set to the deployment name you chose when you deployed the test-embedding-ada-002 (Version 2) model
    )

    res = index.query(
                    vector=xq, 
                    top_k=top, 
                    include_metadata=True
                    )

    data = res.to_dict()
    # Create an empty list to store the transformed data
    transformed_data = []

    # Iterate over the matches and extract relevant information
    for match in data['matches']:
        metadata = match.get('metadata', {})
        row = {
            'id': match['id'],
            'score': match['score'],
            **metadata
        }
        transformed_data.append(row)

    # Create a DataFrame from the transformed data
    df = pd.DataFrame(transformed_data)
    # print(df.dtypes)
    df['date'] = df['date'].astype(str)
    
    return df

In [7]:
search_docs('tariff')

,id,score,date,member,symbol,text,topic
0,80366,0.866429,2011-02-08,Papua New Guinea,WT/TPR/S/239/Rev.1,18. The tariff is PNG's main trade policy inst...,3. trade policies and practices > imports > ta...
1,38993,0.865700,2019-03-06,Chinese Taipei,WT/TPR/S/377/Rev.1,3.2. The tariff is one of the main trade polic...,3. trade policies and practices > other
2,29719,0.861814,2016-02-25,Haiti,WT/TPR/S/327/Rev.1,8. Tariffs are still among Haiti's principal t...,other
3,29806,0.859856,2016-02-25,Haiti,WT/TPR/S/327/Rev.1,3.1. Tariffs are still Haiti's principal trade...,3. trade policies and practices > other
4,20096,0.859657,2017-01-06,"Korea, Republic of",WT/TPR/S/346/Rev.1,3.2. The tariff remains one of the main trade ...,3. trade policies and practices > other


In [10]:
##### ChatGPT augmented generative question answering
# def get_completion(prompt, model="gpt-35-turbo"):
def get_completion(prompt, model="gpt-35-turbo-16k"):
    messages = [{"role": "system", "content":  "You are a Q&A assistant." },
                {"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        # temperature=0.8, # this is the degree of randomness of the model's output
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens=2000,
        # top_p=0.95,
        # frequency_penalty=0,
        # presence_penalty=0,
        # stop=None
    )
    # return response.choices[0].message["content"]
    return response.choices[0].message.content.strip('., ')

get_completion('what is wto')

In [38]:
# prompt with context
limit = 10000
def retrieve(query):
    xq = get_embedding(
        query,
        model='test-embedding-ada-002'
    )

    # retrieve from Pinecone
    # xq = res['data'][0]['embedding']

    # get relevant contexts
    # res = index.query(xq, top_k=100, include_metadata=True)

    res = index.query(
                    vector=xq, 
                    top_k=100, 
                    include_metadata=True
                    )



    contexts = [
        '('+ x['metadata']['member'] + ') (' + x['metadata']['symbol'] + ') paragraph ' + x['metadata']['text'][0:400] for x in res['matches']
    ]

    # build our prompt with the retrieved contexts included
    prompt_start = (
        # "Answer the following question based on the context provided below, which are the relevant paragraphs from the WTO member's TPR reports. Paragraph start with a country name, a document symbol and a paragraph number. \n\n"+
        # "In the answers, be sure to add document symbols and paragraph numbers as references for finding in the answer. \n\n"+
        # "If you don't know the answer, just say that you don't know. Don't try to make up an answer. Do not answer beyond this context. \n\n"+
        # "Context:\n"
        "Answer the following question based on the context provided below, which are the relevant paragraphs from the WTO member's TPR reports.  \n\n" +
        "Paragraph start with a country name, a document symbol and a paragraph number. \n\n" +
        "In your answers, be sure to add document symbols and paragraph numbers as references for finding in the answer. \n\n" +
        "If you don't know the answer, just say that you don't know. Don't try to make up an answer. Do not answer beyond this context. \n\n" +
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {query}\n\nAnswer:"
    )
    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    return prompt


In [39]:
prompt = retrieve(query)

In [21]:
model='gpt-35-turbo-16k'
query='How WTO members protect biodiversity through their trade policy?'
prompt = f"""
            Answer the following question.
            If you don't know the answer, just say that you don't know. 
            ---
            QUESTION: {query}   
        """
chatgpt = get_completion(prompt, model)     

In [16]:
chatgpt

'WTO members protect biodiversity through their trade policy by implementing measures such as trade restrictions, regulations, and agreements that aim to prevent the trade of endangered species, illegal wildlife products, and unsustainable natural resources. They may also promote sustainable trade practices, encourage the use of eco-friendly technologies, and support conservation efforts. However, specific measures can vary among countries and are subject to ongoing negotiations and discussions within the WTO'